In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import torchvision
import time
from torchvision import transforms,models,datasets
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
from collections import OrderedDict
from PIL import Image
import seaborn as sns
import numpy as np 
import pandas as pd 
import json

In [18]:
url = 'https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv'
data = pd.read_csv(url)
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [19]:
data.loc[data['class'] == 0]['tweet']

85       "@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1...
89       "@CB_Baby24: @white_thunduh alsarabsss" hes a ...
110      "@DevilGrimz: @VigxRArts you're fucking gay, b...
184      "@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPL...
202      "@NoChillPaz: "At least I'm not a nigger" http...
                               ...                        
24576                  this guy is the biggest faggot omfg
24685    which one of these names is more offensive kik...
24751           you a pussy ass nigga and I know it nigga.
24776                                   you're all niggers
24777    you're such a retard i hope you get type 2 dia...
Name: tweet, Length: 1430, dtype: object

In [20]:
l = data.groupby('class').size()
l

class
0     1430
1    19190
2     4163
dtype: int64

In [21]:
no_offence=data.loc[data['class'] == 2]['tweet'].copy().reset_index(drop=True)
offence=data.loc[data['class'] == 0]['tweet'].copy().reset_index(drop=True)
X = offence[:1000].append(no_offence[:1000], ignore_index=True)
no_offenceY=data.loc[data['class'] == 2]['class'].copy().reset_index(drop=True)
offenceY=data.loc[data['class'] == 0]['class'].copy().reset_index(drop=True)
Y = offenceY[:1000].append(no_offenceY[:1000], ignore_index=True)
X.head(50)
# Y=data['class'][:1430].copy()
# # l = X.groupby('class').size()
# data[:1430].groupby('class').size()

0     "@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1...
1     "@CB_Baby24: @white_thunduh alsarabsss" hes a ...
2     "@DevilGrimz: @VigxRArts you're fucking gay, b...
3     "@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPL...
4     "@NoChillPaz: "At least I'm not a nigger" http...
5     "@NotoriousBM95: @_WhitePonyJr_ Ariza is a sna...
6     "@RTNBA: Drakes new shoes that will be release...
7     "@TheoMaxximus: #GerrysHalloweenParty http://t...
8     "@ashlingwilde: @ItsNotAdam is bored supposed ...
9     "@bigbootybishopp: @white_thunduh lassen cc , ...
10    "@jayswaggkillah: Jackies a retard #blondeprob...
11    "@jgabsss: Stacey Dash won &#128166; http://t....
12    "Don't worry about the nigga you see, worry ab...
13    "Hey go look at that video of the man that fou...
14    "Let's kill cracker babies!". WTF did I just h...
15    "My grandma used to call me a porch monkey all...
16    "Nah its You @NoMeek_JustMilz: &#128514;&#1285...
17    "Our people". Now is the time for the Arya

In [22]:
from string import punctuation
for i in range(len(X)):
  X[i] = X[i].lower() # lowercase, standardize
  x= X[i].split()
  X[i]=" ".join([c for c in x if '@' not in c])
  X[i] = ''.join([c for c in X[i] if c not in punctuation])

X.head(50)

0                                          queer gaywad
1     alsarabsss hes a beaner smh you can tell hes a...
2     youre fucking gay blacklisted hoe holding out ...
3     lmfaoooo i hate black people httpstcornvd2nlcd...
4      at least im not a nigger httptcorgja7cfoit lmfao
5     ariza is a snake and a coward but at least he ...
6     drakes new shoes that will be released by nike...
7     gerryshalloweenparty httptco3ycrsrnjhc hallowe...
8     is bored supposed to be cute you faggot sometimes
9     lassen cc  the coach wants me i feel it youll ...
10    jackies a retard blondeproblems at least i can...
11    stacey dash won 128166 httptcopdlg46rjol badde...
12    dont worry about the nigga you see worry about...
13    hey go look at that video of the man that foun...
14     lets kill cracker babies wtf did i just hear wow
15    my grandma used to call me a porch monkey all ...
16    nah its you 128514128514128514128514 yo i thou...
17    our people now is the time for the aryan r

In [23]:
"@"

'@'

In [0]:
words = ' '.join(X).split()

In [0]:
from collections import Counter

counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)


In [26]:
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
print(vocab_to_int)

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for x in X:
  reviews_ints.append([vocab_to_int[word] for word in x.split()])
  
print(reviews_ints)

{'the': 1, 'a': 2, 'i': 3, 'you': 4, 'to': 5, 'and': 6, 'is': 7, 'in': 8, 'of': 9, 'that': 10, 'trash': 11, 'like': 12, 'on': 13, 'be': 14, 'for': 15, 'it': 16, 'my': 17, 'are': 18, 'your': 19, 'faggot': 20, 'me': 21, 'bitch': 22, 'this': 23, 'u': 24, 'they': 25, 'all': 26, 'with': 27, 'have': 28, 'im': 29, 'ass': 30, 'dont': 31, 'just': 32, 'but': 33, 'white': 34, 'not': 35, 'he': 36, 'rt': 37, 'so': 38, 'up': 39, 'was': 40, 'nigga': 41, 'its': 42, 'at': 43, 'we': 44, 'no': 45, 'fuck': 46, 'nigger': 47, 'get': 48, 'if': 49, 'lol': 50, 'what': 51, 'can': 52, 'yellow': 53, 'out': 54, 'about': 55, 'do': 56, 'his': 57, 'fag': 58, 'when': 59, 'them': 60, 'yankees': 61, 'bird': 62, 'fucking': 63, 'or': 64, 'youre': 65, 'one': 66, 'as': 67, 'amp': 68, 'got': 69, '9733': 70, 'people': 71, 'some': 72, 'from': 73, 'know': 74, 'only': 75, 'hate': 76, 'go': 77, 'who': 78, 'thats': 79, 'how': 80, 'would': 81, 'shit': 82, 'da': 83, 'an': 84, 'love': 85, 'say': 86, 'these': 87, 'black': 88, 'man': 8

In [27]:
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 32


In [0]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [29]:
seq_length = 10

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features.shape)

(2000, 10)


In [0]:
Y = Y.replace(2, 1)

Y = np.asarray(Y)


In [0]:
split_frac = 0.8
from sklearn.model_selection import train_test_split

## split data into training, validation, and test data (features and labels, x and y)

# split_idx = int(len(features)*split_frac)
# train_x, remaining_x = features[:split_idx], features[split_idx:]
# train_y, remaining_y = Y[:split_idx], Y[split_idx:]
# test_idx = int(len(remaining_x)*0.5)
# val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
# val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]
X_train, X_test, y_train, y_test = train_test_split(features, Y, test_size=0.2, random_state=42)#####
## print out the shapes of your resultant feature data


In [52]:
uniqueValues, occurCount = np.unique(y_train, return_counts=True)
print(uniqueValues, occurCount)
for i in X_train :
  print(i)

[0 1] [801 799]
[  37 3440  147    2   20   15   35 1679   57  226]
[   0    0    0    0    0  528   20  415    2 2413]
[   3  101  111 1116 1113   15   95    6  616 3156]
[1242 2953 2954 2955 2956  356 2957 1548 2958 2959]
[  79  320  770  109   19  142    8   23 1444    4]
[3724  458    4 3725   10  100 3726 3727    8   19]
[ 128    1  881   36 1846   16    5 1984   16   99]
[ 103 2549   24   10    7    2  384    4    2   47]
[   0    0    0    0    0 3544    3   69  978   50]
[   3  276 1473 1824    5 4210    1   53 1825 4211]
[2815  985    6 2816   18  315  354 2817  655   60]
[  46   10  107   79 1050   51 3034  151 3035  783]
[ 166  733  254 5367  130  239  111    9 1611    1]
[3398  678  679  597 3399  678  679  118 3400 1669]
[  78    7  112   37  138 1264   10  912 2146  913]
[4625    3  158 4626    8 4627   17 4628   18 1889]
[ 140 2431  317  626  184   65   63  148   49    4]
[   0    0    0    0    0 2094 1245  902 1246 2095]
[  4 470   4  90  12   2 198 300 168 170]
[  18 

In [0]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 25
X_train = torch.stack([torch.Tensor(i) for i in X_train]) # transform to torch tensors
y_train = torch.stack([torch.Tensor([i]) for i in y_train])
X_test = torch.stack([torch.Tensor(i) for i in X_test]) # transform to torch tensors
y_test = torch.stack([torch.Tensor([i]) for i in y_test])
my_dataset = TensorDataset(X_train,y_train) # create your datset
my_dataset_val = TensorDataset(X_test,y_test) # create your datset
train_loader = DataLoader(my_dataset,batch_size=batch_size) #
val_loader = DataLoader(my_dataset_val,batch_size=batch_size)

# create Tensor datasets
# train_data = TensorDataset(X_train, y_train)
# valid_data = TensorDataset(X_test, y_test)
# # test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# # dataloaders
#batch_size = 25

# # make sure the SHUFFLE your training data
# train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last=True)
# valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size,drop_last=True)
# test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,drop_last=True)

In [62]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [0]:
import torch.nn as nn

class SentimentRNN(nn.Module):

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)
        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [64]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 1000
hidden_dim = 500
n_layers = 1

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(5834, 1000)
  (lstm): LSTM(1000, 500, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=500, out_features=1, bias=True)
  (sig): Sigmoid()
)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [0]:
# loss and optimization functions
lr=0.0003

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters())


In [66]:
# training params

epochs = 30 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 20
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)
    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()
        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()
                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())
                
            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([25, 1])) that is different to the input size (torch.Size([25])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


NameError: ignored

In [0]:
# Get test data loss and accuracy
from sklearn.metrics import f1_score

test_losses = [] # track loss
num_correct = 0

f1=0
# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)
    f1 += f1_score(labels.cpu(), output.cpu() > 0.5, average='weighted')


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
f1= f1 / len(test_loader.dataset)

print("Test accuracy: {:.3f}".format(test_acc))
print("f1 score: {:.3f}".format(f1))


Test loss: 2.409
Test accuracy: 0.450
f1 score: 0.007


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
test_review_neg = 'stfu you nigga'
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(test_review_neg)
# test sequence padding
seq_length=200
features = pad_features(test_ints, seq_length)
feature_tensor = torch.from_numpy(features)
def predict(net, test_review, sequence_length=200):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    print(output.squeeze())
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("no hate detected!")
    else:
        print("Hate speech detected.")
predict(net, test_review_neg, seq_length)

        

tensor(2.8993e-06, device='cuda:0', grad_fn=<SqueezeBackward0>)
Prediction value, pre-rounding: 0.000003
Hate speech detected.
